In [1]:
import pandas as pd

Reading the data file

In [55]:
#exception handle file not found in read_csv
try:
    data= pd.read_csv("102317137-data.csv")
    display(data)
except FileNotFoundError:
    print("Error: This file does not exist")

,Fund Name,P1,P2,P3,P4,P5
0,M1,0.69,0.48,6.0,42.7,12.47
1,M2,0.79,0.62,5.6,57.4,16.10
2,M3,0.65,0.42,5.9,39.9,11.72
3,M4,0.91,0.83,3.7,58.4,15.96
4,M5,0.62,0.38,3.1,59.8,15.98
5,M6,0.63,0.40,5.1,42.2,12.08
6,M7,0.68,0.46,3.7,66.0,17.71
7,M8,0.85,0.72,3.5,47.6,13.17


Dataset is loaded, now 

In [56]:
(data.dtypes[1])
data.columns[1:]

C:\Users\GC\AppData\Local\Temp\ipykernel_19048\1253759217.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (data.dtypes[1])


Index(['P1', 'P2', 'P3', 'P4', 'P5'], dtype='object')

Now we need to implement topsis

Algorithm:
Step 1: for every column, we need to calculate the divisor, which is sum of square of every element of column's root, and then divide every element of the column with it,make the changes in place

In [57]:
import numpy as np
for col in data.columns[1:]:
    print((data[col]**2).sum())

4.317
2.5145
177.42000000000002
22099.26
1695.1707000000001


In [58]:
for col in data.columns[1:]:
    sum_of_squares=(data[col]**2).sum()
    divisor= np.sqrt(sum_of_squares)
    data[col]=data[col]/divisor
display(data)
        

,Fund Name,P1,P2,P3,P4,P5
0,M1,0.332092,0.302702,0.450453,0.287236,0.302872
1,M2,0.380221,0.390990,0.420423,0.386121,0.391038
2,M3,0.312840,0.264864,0.442946,0.268401,0.284656
3,M4,0.437976,0.523422,0.277780,0.392847,0.387638
4,M5,0.298401,0.239639,0.232734,0.402265,0.388124
5,M6,0.303214,0.252252,0.382885,0.283873,0.293400
6,M7,0.327279,0.290090,0.277780,0.443971,0.430142
7,M8,0.409099,0.454053,0.262765,0.320198,0.319874


In [59]:
weights=[1,1,1,2,2]

In [60]:
#Multiply every column by the weight
i=0
for col in data.columns[1:]:
    data[col]*=weights[i]
    i+=1
print(data)

  Fund Name        P1        P2        P3        P4        P5
0        M1  0.332092  0.302702  0.450453  0.574472  0.605745
1        M2  0.380221  0.390990  0.420423  0.772241  0.782076
2        M3  0.312840  0.264864  0.442946  0.536802  0.569313
3        M4  0.437976  0.523422  0.277780  0.785695  0.775276
4        M5  0.298401  0.239639  0.232734  0.804530  0.776247
5        M6  0.303214  0.252252  0.382885  0.567745  0.586800
6        M7  0.327279  0.290090  0.277780  0.887943  0.860284
7        M8  0.409099  0.454053  0.262765  0.640395  0.639748


Normalization Done
Step3: Find the best and the worst value, for this we need the impacts
If impact is +ve the highest value will be the best, if impact is -ve the lowest value will be the best

In [62]:
impacts=['+','+','-','+','-']

In [63]:
best_values=[]
worst_values=[]
#Iterate the data frame column to find the best and worst values
i=0
for col in data.columns[1:]:
    if(impacts[i]=='+'):
        best_values.append(data[col].max())
        worst_values.append(data[col].min())
    else:
        best_values.append(data[col].min())
        worst_values.append(data[col].max())
    i+=1
display(best_values)
display(worst_values)
    

[np.float64(0.4379760555658443),
 np.float64(0.5234223613711745),
 np.float64(0.23273430684125948),
 np.float64(0.8879429542128392),
 np.float64(0.5693127219361015)]

[np.float64(0.2984012686272785),
 np.float64(0.239639153398851),
 np.float64(0.45045349711211513),
 np.float64(0.5368018768650346),
 np.float64(0.8602839851099281)]

Step 4: Calculate the Euclidean distance from each alternative to:
Ideal best (S+)
Ideal worst (S-)
for this we need to iterate every row i, and for every column we need to do (best_values[i]-data[i][j])^2 .sum() and then underroot to get S+ for that row
similarly worst_values[i]-data[i][j] for S-

In [64]:

S_plus=[]
S_minus=[]

#index of the row of data
for index,row in data.iterrows():
    #reset sum for each row
    sum_best=0
    sum_worst=0
    #i is the counter to iterable, and col is column of dataframe from 1 to last
    i=0
    for col in (data.columns[1:]):
        sum_best+=(row[col]-best_values[i])**2
        sum_worst+=(row[col]-worst_values[i])**2
        i+=1
    S_plus.append(np.sqrt(sum_best))
    S_minus.append(np.sqrt(sum_worst))

print("S+ (distance from best): ",S_plus)
print("S- (distance from worst):",S_minus)


S+ (distance from best):  [np.float64(0.4548865769124738), np.float64(0.3387570035144252), np.float64(0.500000265802208), np.float64(0.2343170392457914), np.float64(0.3870317964944941), np.float64(0.46590981730825226), np.float64(0.39165222204976946), np.float64(0.2697940871665372)]
S- (distance from worst): [np.float64(0.26706030141206993), np.float64(0.30340001937806493), np.float64(0.29251568665564726), np.float64(0.44609917283317035), np.float64(0.35516509350060804), np.float64(0.2837227331649091), np.float64(0.3955950726128699), np.float64(0.3909245530208543)]


Step5: Calculate the performace score
Formula: S-/S-+S+

In [54]:
performance=[]
for i in range(len(S_plus)):
    performance.append([S_minus[i]/(S_plus[i]+S_minus[i])])
print(performance)
data["Topsis Score"]=performance
data["Rank"]= data["Topsis Score"].rank(ascending=True).astype(int)
display(data)

[[np.float64(0.22034893417250784)], [np.float64(0.41713314633021686)], [np.float64(0.1815589934305014)], [np.float64(0.8256067583378562)], [np.float64(0.43465948091476375)], [np.float64(0.22919837088818787)], [np.float64(0.4407816422050524)], [np.float64(0.747272482285411)]]


,Fund Name,P1,P2,P3,P4,P5,Topsis Score,Rank
0,M1,0.332092,0.605404,0.900907,0.287236,0.302872,[0.22034893417250784],2
1,M2,0.380221,0.781980,0.840847,0.386121,0.391038,[0.41713314633021686],4
2,M3,0.312840,0.529729,0.885892,0.268401,0.284656,[0.1815589934305014],1
3,M4,0.437976,1.046845,0.555559,0.392847,0.387638,[0.8256067583378562],8
4,M5,0.298401,0.479278,0.465469,0.402265,0.388124,[0.43465948091476375],5
5,M6,0.303214,0.504503,0.765771,0.283873,0.293400,[0.22919837088818787],3
6,M7,0.327279,0.580179,0.555559,0.443971,0.430142,[0.4407816422050524],6
7,M8,0.409099,0.908106,0.525529,0.320198,0.319874,[0.747272482285411],7
